In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import numpy as np
import pandas as pd
import os
import bz2
import nltk
from nltk.corpus import stopwords
#from nltk import word_tokenize
from nltk.tokenize import word_tokenize
#libraries for lemmatization
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet
#nltk.download('averaged_perceptron_tagger')
#nltk.download('all')

In [ ]:
os.listdir('/content/drive/MyDrive/sentitment analysis project')

['test.ft.txt.bz2',
 'train.ft.txt.bz2',
 'reduced_train.csv',
 'reduced_test.csv',
 'new_train.csv',
 'new_test.csv']

In [ ]:
train_data = bz2.BZ2File('/content/drive/MyDrive/sentitment analysis project/train.ft.txt.bz2')
test_data  = bz2.BZ2File('/content/drive/MyDrive/sentitment analysis project/test.ft.txt.bz2')

In [ ]:
def load_and_extract(data):
    texts, labels = [], []
    for line in data:
        x=line.decode('utf-8') #decodes the binary into strings
        labels.append(int(x[9])-1)
        texts.append(x[10:].strip())
    print('complete')
    return np.array(labels), texts

In [ ]:
train_labels, train_texts = load_and_extract(train_data)
test_labels, test_texts = load_and_extract(test_data)

complete
complete


In [ ]:
print(f'The length of the texts dataset for training is : {len(train_texts)}')
print(f'The length of the label  dataset for training is : {len(train_labels)}')
print(f'The length of the texts dataset for testing is : {len(test_texts)}')
print(f'The length of the label dataset for testing is : {len(test_labels)}')

The length of the texts dataset for training is : 3600000
The length of the label  dataset for training is : 3600000
The length of the texts dataset for testing is : 400000
The length of the label dataset for testing is : 400000


In [ ]:
train_df = pd.DataFrame({'Reviews':train_texts, 'Sentiments':train_labels})
test_df = pd.DataFrame({'Reviews':test_texts, 'Sentiments':test_labels})
test_df.head()

,Reviews,Sentiments
0,Great CD: My lovely Pat has one of the GREAT v...,1
1,One of the best game music soundtracks - for a...,1
2,Batteries died within a year ...: I bought thi...,0
3,"works fine, but Maha Energy is better: Check o...",1
4,Great for the non-audiophile: Reviewed quite a...,1


In [ ]:
#check for imbalance in the labels
balance_check = pd.DataFrame(train_df['Sentiments'].value_counts().reset_index())
balance_check.columns = ['sentiments', 'frequency']
balance_check['sentiments'] = balance_check['sentiments'].map({1:'positive', 0:'negative'})
balance_check

,sentiments,frequency
0,positive,1800000
1,negative,1800000


In [ ]:
#reduce the dataset for training to avoid the cost of computatiion
train_df1 = train_df[train_df['Sentiments']==1] # rows with positive sentiments
train_df2 = train_df[train_df['Sentiments']==0]   #rows with negative sentiments
#concatenate the two data with positive and negative sentiment
train_dff = pd.concat([train_df1.sample(n=15000), train_df2.sample(n=15000)])
train_dff = train_dff.sample(frac=1)
train_dff.reset_index(inplace=True)
train_dff.drop(columns='index', inplace=True)
train_dff.head()


,Reviews,Sentiments
0,Loved this game: Oh the nostalgia! This was on...,1
1,"No other movie is yet made, that beats this on...",1
2,This one's a keeper..: One of the songs on thi...,1
3,Really Pretentious: this movie is 2.5 hours lo...,0
4,Good beat'em up for fans of the series: This i...,1


In [ ]:
train_dff['Sentiments'].value_counts()

1    100000
0    100000
Name: Sentiments, dtype: int64

In [ ]:
#reduce the dataset for training to avoid the cost of computatiion
test_df1 = test_df[test_df['Sentiments']==1] # rows with positive sentiments
test_df2 = test_df[test_df['Sentiments']==0]   #rows with negative sentiments
#concatenate the two data with positive and negative sentiment
test_dff = pd.concat([test_df1.sample(n=2500), train_df2.sample(n=2500)])
test_dff = test_dff.sample(frac=1)
test_dff.reset_index(inplace=True)
test_dff.drop(columns='index', inplace=True)
test_dff.head()

,Reviews,Sentiments
0,Disappointing: I was surprised to find this bo...,0
1,NOT THE PIT'S: EVERYONE WHO OWENS OR DOES NOT ...,1
2,The ratings on this book speak to how importan...,1
3,Price for single?: What I don't get is the pri...,0
4,Taylor Timer: Great kitchen Timer! Very nice l...,1


In [ ]:
print(f'the reduced training set is :{train_dff.shape}')
print(f'the reduced testing set is :{test_dff.shape}')

the reduced training set is :(200000, 2)
the reduced testing set is :(50000, 2)


In [ ]:
train_dff.to_csv('/content/drive/MyDrive/sentitment analysis project/Train.csv')

In [ ]:
test_dff.to_csv('/content/drive/MyDrive/sentitment analysis project/Test.csv')